In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import os
import talib as tl
from sklearn.preprocessing import LabelBinarizer, StandardScaler
from sklearn_pandas import DataFrameMapper
%matplotlib inline

In [2]:
def fills(df):
    df.fillna(method='ffill', inplace=True)
    df.fillna(method='bfill', inplace=True)
    return df

In [3]:
aapl = fills(pdr.get_data_google('AAPL', '2000-01-01', '2018-01-01'))
spy = fills(pdr.get_data_google('SPY', '2000-01-01', '2018-01-01'))

In [4]:
oopen = aapl['Open'].values
high = aapl['High'].values
low = aapl['Low'].values
close = aapl['Close'].values

In [5]:
runup252 = aapl['Close'].pct_change(252)
aapl['Run Up 252'] = runup252

In [6]:
beta63 = tl.BETA(close, spy['Close'].values, timeperiod=63)
aapl['Beta 63'] = beta63

In [7]:
ema100 = tl.EMA(close, timeperiod=100)
aapl['EMA 100'] = ema100
aapl['EMA 100'] = aapl['EMA 100'].pct_change()

In [8]:
sma100 = tl.SMA(close, timeperiod=100)
aapl['SMA 100'] = sma100
aapl['SMA 100'] = aapl['SMA 100'].pct_change()

In [9]:
sma_mom100 = aapl['SMA 100'] - aapl['SMA 100'].shift(1)
aapl['SMA MOM 100'] = sma_mom100

In [10]:
sp500_sma100 = tl.SMA(spy['Close'].values, timeperiod=100)
aapl['SP500 SMA 100'] = sp500_sma100
aapl['SP500 SMA 100'] = aapl['SP500 SMA 100'].pct_change()

In [11]:
sp500vola = tl.ATR(spy['High'].values, spy['Low'].values, spy['Close'].values, timeperiod=63)
aapl['SP500 Vola 63'] = sp500vola

In [12]:
sharpe_days = 63
daily_ret = aapl['Close'].pct_change()
sharpe63 = np.sqrt(sharpe_days)*(daily_ret.rolling(sharpe_days).mean()/daily_ret.rolling(sharpe_days).std())
aapl['Sharpe 63'] = sharpe63

In [13]:
vola63 = tl.ATR(high, low, close, timeperiod=63)
aapl['Vola 63'] = vola63

In [14]:
return_days = 10
extreme_return = 5
ret = 100*aapl['Close'].pct_change(return_days).shift(-return_days)
aapl[str(return_days) + ' days future return'] = ret

In [15]:
aapl['Weekday'] = aapl.index.dayofweek.map(lambda x: {0: 'Mon', 1: 'Tue', 2: 'Wed', 3: 'Thu', 4: 'Fri'}.get(x))

In [16]:
def to_class(val, extreme):
    if val < -extreme:
        return 'Neg'
    elif val > extreme:
        return 'Pos'
    else:
        return 'Neutral'
aapl['Return class'] = aapl[str(return_days) + ' days future return'].apply(lambda val: to_class(val, extreme_return))
aapl['Return class'].head()

Date
2002-02-11        Neg
2002-02-12        Neg
2002-02-13        Neg
2002-02-14    Neutral
2002-02-15    Neutral
Name: Return class, dtype: object

In [17]:
aapl.tail(6)

,Open,High,Low,Close,Volume,Run Up 252,Beta 63,EMA 100,SMA 100,SMA MOM 100,SP500 SMA 100,SP500 Vola 63,Sharpe 63,Vola 63,10 days future return,Weekday,Return class
Date,,,,,,,,,,,,,,,,,
2017-12-21,174.17,176.02,174.10,175.01,20949896,0.495045,0.141154,0.001409,0.001530,-0.000043,0.000797,1.454152,1.690945,2.485687,NaN,Thu,Neutral
2017-12-22,174.68,175.42,174.50,175.01,16349444,0.504945,0.139007,0.001379,0.001094,-0.000436,0.000789,1.442816,1.808430,2.460835,NaN,Fri,Neutral
2017-12-26,170.80,171.47,169.68,170.57,33185536,0.463869,0.138846,0.000813,0.000917,-0.000177,0.000795,1.429915,1.263734,2.506377,NaN,Tue,Neutral
2017-12-27,170.10,170.78,169.71,170.60,21498213,0.454887,0.137480,0.000800,0.000868,-0.000049,0.000782,1.418646,1.187278,2.483577,NaN,Wed,Neutral
2017-12-28,171.00,171.85,170.48,171.08,16480187,0.465228,0.138857,0.000841,0.000749,-0.000119,0.000785,1.405652,1.294132,2.465901,NaN,Thu,Neutral
2017-12-29,170.52,170.59,169.22,169.23,25999922,0.449756,0.142824,0.000601,0.000558,-0.000191,0.000768,1.413657,1.098579,2.456284,NaN,Fri,Neutral


In [18]:
aapl.drop(['Open', 'High', 'Low', 'Close', 'Volume', str(return_days) + ' days future return'], axis=1, inplace=True)
aapl.dropna(inplace=True)

### Resample umbalanced data and Train-test split

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

In [20]:
train_index = int(len(aapl) * 0.75)
aapl_train = aapl.iloc[:train_index]
aapl_test = aapl.iloc[train_index:]

In [21]:
df_majority = aapl_train[aapl_train['Return class']=='Pos']
df_minority = aapl_train[aapl_train['Return class']=='Neg']
df_other = aapl_train[aapl_train['Return class']=='Neutral']

df_minority_upsampled = resample(df_minority, 
                                 replace=True,
                                 n_samples=len(df_majority))

aapl_train = pd.concat([df_majority, df_minority_upsampled, df_other])
 
aapl_train['Return class'].value_counts()

Neutral    1477
Pos         900
Neg         900
Name: Return class, dtype: int64

In [22]:
X_train = aapl_train.drop('Return class', axis=1)
X_test = aapl_test.drop('Return class', axis=1)
y_train = aapl_train['Return class']
y_test = aapl_test['Return class']

In [23]:
trafos = [([col], StandardScaler()) for col in aapl.columns[:-2].tolist()]
trafos = [('Weekday', LabelBinarizer())] + trafos
#trafos += [('Return class', LabelBinarizer())]

In [24]:
trafos

[('Weekday', LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)),
 (['Run Up 252'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['Beta 63'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['EMA 100'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['SMA 100'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['SMA MOM 100'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['SP500 SMA 100'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['SP500 Vola 63'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['Sharpe 63'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['Vola 63'], StandardScaler(copy=True, with_mean=True, with_std=True))]

In [25]:
mapper = DataFrameMapper(trafos, df_out=True, default=None)

In [26]:
X_train = mapper.fit_transform(X_train)
X_test = mapper.transform(X_test)
X_train.head()

,Weekday_Fri,Weekday_Mon,Weekday_Thu,Weekday_Tue,Weekday_Wed,Run Up 252,Beta 63,EMA 100,SMA 100,SMA MOM 100,SP500 SMA 100,SP500 Vola 63,Sharpe 63,Vola 63
Date,,,,,,,,,,,,,,
2003-04-11,1.0,0.0,0.0,0.0,0.0,-1.648688,-1.172853,-1.399080,-1.262117,-1.965042,-0.514700,0.273404,-1.233005,-1.241456
2003-04-15,0.0,0.0,0.0,1.0,0.0,-1.654089,-1.218892,-1.232036,-1.016568,0.533144,-0.346298,0.269093,-1.047933,-1.241888
2003-04-16,0.0,0.0,0.0,0.0,1.0,-1.671541,-1.235600,-1.290534,-1.123519,-0.858752,-0.251491,0.253658,-1.052300,-1.241280
2003-04-17,0.0,0.0,1.0,0.0,0.0,-1.662438,-1.163626,-1.348229,-1.372987,-1.979331,-0.604898,0.260539,-1.165342,-1.241226
2003-04-21,0.0,1.0,0.0,0.0,0.0,-1.644001,-1.211125,-1.334130,-1.268014,0.807575,-0.646041,0.249078,-1.009248,-1.241990


In [36]:
X_test.head()

,Weekday_Fri,Weekday_Mon,Weekday_Thu,Weekday_Tue,Weekday_Wed,Run Up 252,Beta 63,EMA 100,SMA 100,SMA MOM 100,SP500 SMA 100,SP500 Vola 63,Sharpe 63,Vola 63
Date,,,,,,,,,,,,,,
2014-04-14,0.0,1.0,0.0,0.0,0.0,-0.509072,0.933304,-0.644398,-0.534931,0.441291,-0.024650,-0.077379,-0.838435,0.841452
2014-04-15,0.0,0.0,0.0,1.0,0.0,-0.552657,0.807982,-0.695996,-0.567263,-0.272037,0.054494,-0.052529,-1.063445,0.847656
2014-04-16,0.0,0.0,0.0,0.0,1.0,-0.436140,0.798774,-0.678381,-0.528658,0.285735,0.173395,-0.049184,-1.219517,0.840419
2014-04-17,0.0,0.0,1.0,0.0,0.0,-0.356125,0.791786,-0.591840,-0.530108,-0.029210,0.117138,-0.060047,-1.076060,0.840377
2014-04-21,0.0,1.0,0.0,0.0,0.0,-0.322399,0.789259,-0.503009,-0.477999,0.391907,0.105044,-0.081773,-0.776939,0.838157


# ML model

In [27]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.grid_search import GridSearchCV

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

C:\DEV\Anaconda3\envs\trading\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\DEV\Anaconda3\envs\trading\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [28]:
def fit_and_print(grid):
    grid.fit(X_train, y_train)
    best_estimator = grid.best_estimator_
    print('Best estimator:\n', str(best_estimator))
    
    y_pred = best_estimator.predict(X_test)
    print('Accuracy score:', str(accuracy_score(y_test, y_pred)))
    print('Classification report\n', classification_report(y_test, y_pred))
    print('Confusion matrix\n', str(confusion_matrix(y_test, y_pred)))

### K nearest

In [29]:
params = {'n_neighbors': np.arange(5,50,3),
          'metric': ['euclidean', 'l2', 'manhattan', 'cityblock'],
          'leaf_size': np.arange(5,40,5), 
          'algorithm' : ['auto']}
grid = GridSearchCV(estimator=KNeighborsClassifier(n_jobs=4), param_grid=params, scoring='f1_weighted')

In [30]:
# fit_and_print(grid)

Best estimator:
 KNeighborsClassifier(algorithm='auto', leaf_size=5, metric='manhattan',
           metric_params=None, n_jobs=4, n_neighbors=14, p=2,
           weights='uniform')
Accuracy score: 0.505869797225
Classification report
              precision    recall  f1-score   support

        Neg       0.09      0.42      0.15        77
    Neutral       0.76      0.62      0.68       707
        Pos       0.10      0.01      0.01       153

avg / total       0.59      0.51      0.53       937

Confusion matrix
 [[ 32  44   1]
 [258 441   8]
 [ 53  99   1]]


### Random Forest

In [37]:
params = {'n_estimators': np.arange(5,20,2),
          'max_features': np.arange(2,10),
          'min_samples_split': [2,3],
          'min_samples_leaf': [2,3]}
grid = GridSearchCV(estimator=RandomForestClassifier(n_jobs=6, class_weight='balanced_subsample'), param_grid=params, scoring='f1_micro')
# grid = GridSearchCV(estimator=RandomForestClassifier(n_jobs=6, class_weight='balanced_subsample'), param_grid=params, scoring='f1_weighted')

In [38]:
fit_and_print(grid)

Best estimator:
 RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample',
            criterion='gini', max_depth=None, max_features=2,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=2,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=5, n_jobs=6, oob_score=False, random_state=None,
            verbose=0, warm_start=False)
Accuracy score: 0.532550693703
Classification report
              precision    recall  f1-score   support

        Neg       0.06      0.16      0.09        77
    Neutral       0.77      0.66      0.71       707
        Pos       0.13      0.11      0.12       153

avg / total       0.61      0.53      0.57       937

Confusion matrix
 [[ 12  37  28]
 [154 470  83]
 [ 33 103  17]]


### Neural net

In [33]:
from sklearn.neural_network import MLPClassifier

In [34]:
params = {'alpha': [1e-5],
          'hidden_layer_sizes': [(4,3), (5,3), (8,3), (10,8,3), (10, 5, 3), (5,4,4,3)]}

grid = GridSearchCV(estimator=MLPClassifier(max_iter=2000, batch_size=128), param_grid=params, scoring='f1_weighted')

In [35]:
fit_and_print(grid)

C:\DEV\Anaconda3\envs\trading\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\DEV\Anaconda3\envs\trading\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\DEV\Anaconda3\envs\trading\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\DEV\Anaconda3\envs\trading\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Best estimator:
 MLPClassifier(activation='relu', alpha=1e-05, batch_size=128, beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 4, 4, 3), learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)
Accuracy score: 0.324439701174
Classification report
              precision    recall  f1-score   support

        Neg       0.09      0.69      0.15        77
    Neutral       0.77      0.36      0.49       707
        Pos       0.00      0.00      0.00       153

avg / total       0.59      0.32      0.38       937

Confusion matrix
 [[ 53  24   0]
 [456 251   0]
 [102  51   0]]


C:\DEV\Anaconda3\envs\trading\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
